# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

In [119]:
import pandas as pd
data = pd.read_csv('census_data.csv', sep=',')
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [138]:
def labels(i):
    if i == ' <=50K':
        return 0
    else:
        return 1

In [139]:
from sklearn.model_selection import train_test_split

In [140]:
new_labels = data['income_bracket'].apply(labels)
x_data = data.drop('income_bracket', axis = 1)

In [141]:
X_train, X_test, y_train, y_test = train_test_split(x_data,new_labels,test_size=0.33)
X_train.shape, X_test.shape

((21815, 13), (10746, 13))

In [142]:
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [143]:
import tensorflow as tf

In [144]:
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender',['F','M'])
occ = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size = 1000)
rel = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size = 1000)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size = 1000)
wc = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size = 1000)
edu = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size = 1000)
mar_st = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size = 1000)
n_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size = 1000)

In [145]:
age  = tf.feature_column.numeric_column('age')
edu_num  = tf.feature_column.numeric_column("education_num")
cap_g  = tf.feature_column.numeric_column("capital_gain")
cap_l  = tf.feature_column.numeric_column("capital_loss")
hpw  = tf.feature_column.numeric_column("hours_per_week")

In [146]:
feat_cols = [age, wc, edu, edu_num, mar_st, occ, rel, race, gender, cap_g, cap_l, hpw, n_country]

In [147]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=10,shuffle=True)

In [148]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\134E~1\\AppData\\Local\\Temp\\tmpv5n7_nf9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000052D471E9E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [149]:
model.train(input_fn = input_func, steps = 5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\134E~1\AppData\Local\Temp\tmpv5n7_nf9\model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 91.8292
INFO:tensorflow:loss = 60.70624, step = 101 (1.089 sec)
INFO:tensorflow:global_step/sec: 145.606
INFO:tensorflow:loss = 1234.5852, step = 201 (0.691 sec)
INFO:tensorflow:global_step/sec: 150.502
INFO:tensorflow:loss = 56.751556, step = 301 (0.667 sec)
INFO:tensorflow:global_step/sec: 178.935
INFO:tensorflow:loss = 133.27905, step = 401 (0.552 sec)
INFO:tensorflow:global_step/sec: 191.156
INFO:tensorflow:loss = 48.009373, step = 501 (0.523 sec)
INFO:tensorflow:global_step/sec: 193.93
INFO:tensorflow:loss = 62.764988, step = 601 (0.516 sec)
INFO:tensorflow:global_step/se

In [150]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=10, num_epochs=1, shuffle=False)
results = model.evaluate(eval_input_func)
results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-16-18:08:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\134E~1\AppData\Local\Temp\tmpv5n7_nf9\model.ckpt-2182
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-16-18:08:48
INFO:tensorflow:Saving dict for global step 2182: accuracy = 0.82868046, accuracy_baseline = 0.7602829, auc = 0.87858826, auc_precision_recall = 0.6651671, average_loss = 1.2597748, global_step = 2182, label/mean = 0.23971711, loss = 12.5930605, precision = 0.64932954, prediction/mean = 0.25719684, recall = 0.6203416
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2182: C:\Users\134E~1\AppData\Local\Temp\tmpv5n7_nf9\model.ckpt-2182


{'accuracy': 0.82868046,
 'accuracy_baseline': 0.7602829,
 'auc': 0.87858826,
 'auc_precision_recall': 0.6651671,
 'average_loss': 1.2597748,
 'label/mean': 0.23971711,
 'loss': 12.5930605,
 'precision': 0.64932954,
 'prediction/mean': 0.25719684,
 'recall': 0.6203416,
 'global_step': 2182}

In [151]:
preds = model.predict(eval_input_func)
predictions = list(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\134E~1\AppData\Local\Temp\tmpv5n7_nf9\model.ckpt-2182
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [135]:
preds_list = list(i['class_ids'][0] for i in predictions)
#print (preds_list)

In [136]:
from sklearn.metrics import classification_report

In [137]:
res = classification_report(y_test, preds_list)
print (res)

             precision    recall  f1-score   support

          0       0.90      0.89      0.89      8172
          1       0.65      0.67      0.66      2574

avg / total       0.84      0.84      0.84     10746

